# Tests unitaires et refactorisation


Les idées explorées dans ce guide sont des piliers fondamentaux dans le contexte global de ce repo en raison de leur importance pour notre objectif ultime : écrire des logiciels meilleurs et plus faciles à maintenir.

Les tests unitaires (et toute forme de tests automatiques, d'ailleurs) sont essentiels à la maintenabilité du logiciel, et donc quelque chose qui ne peut manquer à tout projet de qualité. C'est pour cette raison que ce guide est exclusivement consacré aux aspects des tests automatisés en tant que stratégie clé, pour modifier le code en toute sécurité et le parcourir, dans des versions progressivement meilleures.

Après ce guide, nous aurons mieux compris les éléments suivants : 

* Pourquoi les tests automatisés sont essentiels à la réussite d'un projet 

* Comment les tests unitaires fonctionnent comme une heuristique de la qualité du code 

 
* Quels frameworks et outils sont disponibles pour développer des tests automatisés et mettre en place des portes de qualité

* Tirer parti des tests unitaires pour mieux comprendre le problème du domaine et documenter le code
 Concepts liés aux tests unitaires, tels que le développement piloté par les tests.


Dans les guides précédents, nous avons vu des traits spécifiques à Python et comment nous pouvons les exploiter pour obtenir un code plus maintenable. Nous avons également exploré comment les principes généraux de conception du génie logiciel peuvent être appliqués à Python en utilisant ses particularités. Ici, nous allons également revisiter un concept important de l'ingénierie logicielle, comme le test automatique, mais avec l'utilisation d'outils, certains d'entre eux disponibles dans la bibliothèque standard (comme le module unittest), et d'autres qui sont des packages externes (comme comme pytest). Nous commençons ce voyage en explorant la relation entre la conception de logiciels et les tests unitaires.

## Principes de conception et tests unitaires

Dans cette section, nous allons d'abord aborder les tests unitaires d'un point de vue conceptuel. Nous allons revoir certains des principes de génie logiciel dont nous avons discuté dans le guide précédent pour avoir une idée de la façon dont cela est lié au code propre.

Après cela, nous discuterons plus en détail de la façon de mettre ces concepts en pratique (au niveau du code) et des frameworks et outils que nous pouvons utiliser.

Tout d'abord, nous définissons rapidement en quoi consistent les tests unitaires. Les tests unitaires sont des codes chargés de valider d'autres parties du code. Normalement, n'importe qui serait tenté de dire que les tests unitaires valident le "noyau" de l'application, mais une telle définition considère les tests unitaires comme secondaires, ce qui n'est pas la façon dont ils sont pensés dans ce guide. Les tests unitaires sont essentiels et un composant critique du logiciel et ils doivent être traités avec les mêmes considérations que la logique métier.


Un test unitaire est un morceau de code qui importe des parties du code avec la logique métier, et exerce sa logique, affirmant plusieurs scénarios avec l'idée de garantir certaines conditions. Les tests unitaires doivent avoir certaines caractéristiques, telles que : 

*  Isolement : les tests unitaires doivent être complètement indépendants de tout autre agent externe et doivent se concentrer uniquement sur la logique métier. Pour cette raison, ils ne se connectent pas à une base de données, ils n'exécutent pas de requêtes HTTP, etc. L'isolement signifie également que les tests sont indépendants entre eux : ils doivent pouvoir s'exécuter dans n'importe quel ordre, sans dépendre d'aucun état antérieur.


* Performances : les tests unitaires doivent s'exécuter rapidement. Ils sont destinés à être exécutés plusieurs fois, de manière répétée. 

* Répétabilité : les tests unitaires doivent être capables d'évaluer objectivement l'état du logiciel de manière déterministe. Cela signifie que les résultats obtenus par les tests doivent être reproductibles. Les tests unitaires évaluent l'état du code : si un test échoue, il doit continuer à échouer jusqu'à ce que le code soit réussi. Si un test réussit et qu'aucune modification n'est apportée au code, il devrait continuer à réussir. Les tests ne doivent pas être flous ou aléatoires.

* Auto-validation : L'exécution d'un test unitaire détermine son résultat. Aucune étape supplémentaire ne devrait être requise pour interpréter le test unitaire (beaucoup moins d'interventions manuelles)

Plus concrètement, en Python, cela signifie que nous aurons de nouveaux fichiers *.py où nous allons placer nos tests unitaires, et ils vont être appelés par un outil. Ces fichiers auront des instructions d'importation, pour prendre ce dont nous avons besoin de notre logique métier (ce que nous avons l'intention de tester), et à l'intérieur de ce fichier, nous programmons les tests eux-mêmes. Par la suite, un outil collectera nos tests unitaires et les exécutera, donnant un résultat.


Cette dernière partie est ce que signifie réellement l'auto-validation. Lorsque l'outil appelle nos fichiers, un processus Python sera lancé et nos tests s'exécuteront dessus. Si les tests échouent, le processus se terminera avec un code d'erreur (dans un environnement Unix, il peut s'agir de n'importe quel nombre autre que 0). La norme est que l'outil exécute le test et imprime un point (.) pour chaque test réussi ; un F si le test a échoué (la condition du test n'a pas été satisfaite), et un E s'il y a eu une exception


## Remarque sur les autres formes de tests automatisés

Les tests unitaires sont destinés à vérifier de très petites unités de code, par exemple, une fonction ou une méthode. Nous voulons que nos tests unitaires atteignent un niveau de granularité très détaillé, testant autant de code que possible. Pour tester quelque chose de plus gros, comme une classe, nous ne voudrions pas utiliser uniquement des tests unitaires, mais plutôt une suite de tests, qui est une collection de tests unitaires. Chacun d'eux testera quelque chose de plus spécifique, comme une méthode de cette classe.

Les tests unitaires ne sont pas le seul mécanisme disponible de test automatique, et nous ne devrions pas nous attendre à ce qu'ils détectent toutes les erreurs possibles. Il existe également des tests d'acceptation et d'intégration, tous deux dépassant le cadre de ce guide.

Dans un test d'intégration, nous voulons tester plusieurs composants à la fois. Dans ce cas, nous voulons valider si collectivement, ils fonctionnent comme prévu. Dans ce cas, il est acceptable (plus que cela, souhaitable) d'avoir des effets secondaires, et d'oublier l'isolement, ce qui signifie que nous voudrons émettre des requêtes HTTP, nous connecter aux bases de données, etc. Bien que nous voudrions que nos tests d'intégration s'exécutent réellement comme le ferait le code de production, il y a certaines dépendances que nous voudrions toujours éviter. Par exemple, si votre service se connecte à une autre dépendance externe via Internet, alors cette partie serait effectivement omise.

Supposons que votre application utilise une base de données et se connecte à d'autres services internes. L'application aura différents fichiers de configuration pour différents environnements, et bien sûr, en production, vous aurez la configuration définie pour les services réels. Cependant, pour un test d'intégration, vous voudrez vous moquer de la base de données avec un conteneur Docker spécialement conçu pour ces tests, et cela sera configuré dans un fichier de configuration spécifique. En ce qui concerne les dépendances, vous voudrez vous en moquer avec les services Docker, chaque fois que cela est possible.

La moquerie dans le cadre des tests unitaires sera abordée plus loin dans ce guide. Lorsqu'il s'agit de se moquer des dépendances pour effectuer des tests de composants, cela sera couvert plus tard dans ce repo, Clean Architecture, lorsque nous mentionnons les composants dans le contexte de l'architecture logicielle.

Un test d'acceptation est une forme automatisée de test qui essaie de valider le système du point de vue d'un utilisateur, en exécutant généralement des cas d'utilisation.

Ces deux dernières formes de tests perdent une autre caractéristique intéressante par rapport aux tests unitaires : la vitesse. Comme vous pouvez l'imaginer, ils prendront plus de temps à s'exécuter, et par conséquent ils seront exécutés moins fréquemment.

Dans un bon environnement de développement, le programmeur disposera de l'intégralité de la suite de tests et exécutera des tests unitaires tout le temps, de manière répétée, tout en apportant des modifications au code, en itérant, en refactorisant, etc. Une fois que les modifications sont prêtes et que la demande d'extraction est ouverte, le service d'intégration continue exécutera le build pour cette branche, où les tests unitaires s'exécuteront aussi longtemps que les tests d'intégration ou d'acceptation qui pourraient exister. Inutile de dire que le statut de la construction doit être réussi (vert) avant la fusion, mais la partie importante est la différence entre les types de tests : nous voulons exécuter des tests unitaires tout le temps, et ces tests qui prennent plus de temps moins fréquemment .

Pour cette raison, nous voulons avoir beaucoup de petits tests unitaires, et quelques tests automatisés, stratégiquement conçus pour couvrir autant que possible les endroits où les tests unitaires ne pourraient pas atteindre (l'utilisation de la base de données, par exemple).

Enfin, un mot aux sages. N'oubliez pas que ce repo encourage le pragmatisme.

Outre ces définitions données et les remarques faites sur les tests unitaires au début de la section, le lecteur doit garder à l'esprit que la meilleure solution selon vos critères et votre contexte doit prédominer. Personne ne connaît votre système mieux que vous, ce qui signifie que si, pour une raison quelconque, vous devez écrire un test unitaire qui doit lancer un conteneur Docker pour tester une base de données, allez-y. Comme nous l'avons rappelé à plusieurs reprises tout au long dde ce repo, la praticité bat la pureté.

## Tests unitaires et développement logiciel agile

Dans le développement de logiciels modernes, nous voulons offrir de la valeur en permanence et le plus rapidement possible. La justification de ces objectifs est que plus tôt nous recevons des feedback moins l'impact est important et plus il sera facile de changer. Ce ne sont pas du tout des idées nouvelles ; certains d'entre eux ressemblent à des principes d'il y a des décennies, et d'autres (comme l'idée d'obtenir les feedback des parties prenantes dès que possible et de les répéter) que vous pouvez trouver dans des essais tels que La cathédrale et le bazar (en abrégé CatB).

Par conséquent, nous voulons pouvoir répondre efficacement aux changements, et pour cela, le logiciel que nous écrivons devra changer. Comme je l'ai mentionné dans les guide précédents, nous voulons que notre logiciel soit adaptable, flexible et extensible.

Le code seul (peu importe à quel point il est bien écrit et conçu) ne peut pas nous garantir qu'il est suffisamment flexible pour être modifié, s'il n'y a pas de preuve formelle qu'il continuera à fonctionner correctement après avoir été modifié

Disons que nous concevons un logiciel en suivant les principes SOLID, et dans une partie nous avons en fait un ensemble de composants qui sont conformes au principe ouvert/fermé, ce qui signifie que nous pouvons facilement les étendre sans affecter trop de code existant.

Supposons en outre que le code est écrit d'une manière qui favorise la refactorisation, afin que nous puissions le modifier si nécessaire. Qu'est-ce qui veut dire que lorsque nous effectuons ces changements, nous n'introduisons aucun bug ? Comment savons-nous que les fonctionnalités existantes sont préservées (et qu'il n'y a pas de régressions) ? Vous sentiriez-vous suffisamment confiant pour communiquer cela à vos utilisateurs ? Croiront-ils que la nouvelle version fonctionne comme prévu?

La réponse à toutes ces questions est que nous ne pouvons en être sûrs que si nous en avons une preuve formelle. Et les tests unitaires ne sont que cela : la preuve formelle que le programme fonctionne selon les spécifications

Les tests unitaires (ou automatisés) fonctionnent donc comme un filet de sécurité qui nous donne la confiance nécessaire pour travailler sur notre code. Armés de ces outils, nous pouvons travailler efficacement sur notre code, et c'est donc ce qui détermine en fin de compte la vitesse (ou la capacité) de l'équipe travaillant sur le produit logiciel. Plus les tests sont bons, plus il est probable que nous puissions fournir de la valeur rapidement sans être arrêtés par des bugs de temps en temps

## ests unitaires et conception de logiciels

C'est l'autre face de la médaille lorsqu'il s'agit de la relation entre le code principal et les tests unitaires. Outre les raisons pragmatiques explorées dans la section précédente, cela se résume au fait qu'un bon logiciel est un logiciel testable.

La testabilité (l'attribut de qualité qui détermine la facilité de test du logiciel) n'est pas seulement agréable à avoir, mais un pilote pour un code propre.

Les tests unitaires ne sont pas seulement quelque chose de complémentaire à la base de code principale, mais plutôt quelque chose qui a un impact direct et une réelle influence sur la façon dont le code est écrit. Il y a plusieurs niveaux de cela, dès le début, lorsque nous réalisons qu'au moment où nous voulons ajouter des tests unitaires pour certaines parties de notre code, nous devons le changer (ce qui entraîne une meilleure version de celui-ci), jusqu'à son expression ultime (exploré vers la fin de ce guide) lorsque l'ensemble du code (la conception) est piloté par la façon dont il va être testé via une conception pilotée par les tests.

En partant d'un exemple simple, je vais vous montrer un petit cas d'utilisation dans lequel les tests (et la nécessité de tester notre code) conduisent à des améliorations dans la façon dont notre code finit par être écrit

Dans l'exemple suivant, nous allons simuler un processus qui nécessite l'envoi de métriques à un système externe sur les résultats obtenus à chaque tâche particulière (comme toujours, les détails ne feront aucune différence tant que nous nous concentrons sur le code). Nous avons un objet Process qui représente une tâche sur le problème du domaine, et il utilise un client de métriques (une dépendance externe et donc quelque chose que nous ne contrôlons pas) pour envoyer les métriques réelles à l'entité externe (cela pourrait envoyer des données à syslog , ou statsd, par exemple)

In [ ]:
import logging
import random

logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)


class MetricsClient:
    """3rd-party metrics client"""

    def send(self, metric_name, metric_value):
        if not isinstance(metric_name, str):
            raise TypeError("expected type str for metric_name")

        if not isinstance(metric_value, str):
            raise TypeError("expected type str for metric_value")

        logger.info("sending %s = %s", metric_name, metric_value)


class Process:
    """A job that runs in iterations, and depends on an external object."""

    def __init__(self):
        self.client = MetricsClient()  # A 3rd-party metrics client

    def process_iterations(self, n_iterations):
        for i in range(n_iterations):
            result = self.run_process()
            self.client.send(f"iteration.{i}", str(result))

    def run_process(self):
        return random.randint(1, 100)


if __name__ == "__main__":
    Process().process_iterations(10)

INFO:__main__:sending iteration.0 = 99
INFO:__main__:sending iteration.1 = 85
INFO:__main__:sending iteration.2 = 69
INFO:__main__:sending iteration.3 = 87
INFO:__main__:sending iteration.4 = 30
INFO:__main__:sending iteration.5 = 75
INFO:__main__:sending iteration.6 = 35
INFO:__main__:sending iteration.7 = 99
INFO:__main__:sending iteration.8 = 74
INFO:__main__:sending iteration.9 = 60


Dans la version simulée du client tiers, nous mettons l'exigence que les paramètres fournis doivent être de type chaîne. Par conséquent, si le résultat de la méthode run_process n'est pas une chaîne, nous pouvons nous attendre à ce qu'il échoue, et c'est effectivement le cas.

N'oubliez pas que cette validation ne dépend pas de nous et que nous ne pouvons pas modifier le code, nous devons donc fournir à la méthode des paramètres du type correct avant de continuer. Mais comme il s'agit d'un bogue que nous avons détecté, nous voulons d'abord écrire un test unitaire pour nous assurer que cela ne se reproduira plus. Nous faisons cela pour prouver que nous avons résolu le problème et pour nous protéger contre ce bogue à l'avenir, quel que soit le nombre de fois que le code est modifié.

Il serait possible de tester le code tel quel en se moquant du client de l'objet Process (nous verrons comment faire dans la section Objets simulés, lorsque nous explorerons les outils de test unitaire), mais cela exécute plus de code que ce qui est nécessaire (notez comment la partie que nous voulons tester est imbriquée dans le code). De plus, il est bon que la méthode soit relativement petite, car si ce n'était pas le cas, le test devrait exécuter encore plus de parties indésirables dont nous pourrions également avoir besoin de se moquer. Ceci est un autre exemple de bonne conception (petites fonctions ou méthodes cohésives), qui se rapporte à la testabilité

Enfin, nous décidons de ne pas nous donner trop de peine et de tester uniquement la partie dont nous avons besoin, donc au lieu d'interagir avec le client directement sur la méthode principale, nous déléguons à une méthode wrapper, et la nouvelle classe ressemble à ceci

In [ ]:
import logging
import random

logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)


class MetricsClient:
    """3rd-party metrics client"""

    def send(self, metric_name, metric_value):
        if not isinstance(metric_name, str):
            raise TypeError("expected type str for metric_name")

        if not isinstance(metric_value, str):
            raise TypeError("expected type str for metric_value")

        logger.info("sending %s = %s", metric_name, metric_value)


class WrappedClient:
    """An object under our control that wraps the 3rd party one."""

    def __init__(self):
        self.client = MetricsClient()

    def send(self, metric_name, metric_value):
        return self.client.send(str(metric_name), str(metric_value))


class Process:
    """Same process, now using a wrapper object."""

    def __init__(self):
        self.client = WrappedClient()

    def process_iterations(self, n_iterations):
        for i in range(n_iterations):
            result = self.run_process()
            self.client.send("iteration.{}".format(i), result)

    def run_process(self):
        return random.randint(1, 100)


if __name__ == "__main__":
    Process().process_iterations(10)

INFO:__main__:sending iteration.0 = 25
INFO:__main__:sending iteration.1 = 35
INFO:__main__:sending iteration.2 = 3
INFO:__main__:sending iteration.3 = 46
INFO:__main__:sending iteration.4 = 100
INFO:__main__:sending iteration.5 = 81
INFO:__main__:sending iteration.6 = 52
INFO:__main__:sending iteration.7 = 22
INFO:__main__:sending iteration.8 = 25
INFO:__main__:sending iteration.9 = 84


Dans ce cas, nous avons opté pour la création de notre propre version du client pour les métriques, c'est-à-dire un wrapper autour de la bibliothèque tierce que nous avions. Pour ce faire, nous plaçons une classe qui (avec la même interface) fera la conversion des types en conséquence.

Cette façon d'utiliser la composition ressemble au modèle de conception d'adaptateur (nous explorerons les modèles de conception dans le guide suivant, donc, pour l'instant, ce n'est qu'un message informatif), et comme il s'agit d'un nouvel objet dans notre domaine, il peut avoir son tests unitaires. Avoir cet objet rendra les choses plus simples à tester, mais plus important encore, maintenant que nous l'examinons, nous réalisons que c'est probablement la façon dont le code aurait dû être écrit en premier lieu. Essayer d'écrire un test unitaire pour notre code nous a fait réaliser qu'il nous manquait complètement une abstraction importante.

Maintenant que nous avons séparé la méthode comme il se doit, écrivons le test unitaire réel pour elle. Les détails relatifs au module unittest utilisé dans cet exemple seront explorés plus en détail dans la partie du guide où nous explorons les outils de test et les bibliothèques, mais pour l'instant, la lecture du code nous donnera une première impression sur la façon de le tester , et cela rendra les concepts précédents un peu moins abstraits

In [ ]:
from unittest import TestCase
from unittest.mock import Mock




class TestWrappedClient(TestCase):
    def test_send_converts_types(self):
        wrapped_client = WrappedClient()
        wrapped_client.client = Mock()
        wrapped_client.send("value", 1)

        wrapped_client.client.send.assert_called_with("value", "1")


Mock est un type disponible dans le module unittest.mock, qui est un objet pratique pour poser des questions sur toutes sortes de choses. Par exemple, dans ce cas, nous l'utilisons à la place de la bibliothèque tierce (simulée dans les limites du système, comme indiqué dans la section suivante) pour vérifier qu'elle est appelée comme prévu (et encore une fois, nous ne testez pas la bibliothèque elle-même, seulement qu'elle est appelée correctement). Remarquez comment nous exécutons un appel comme celui de notre objet Process, mais nous nous attendons à ce que les paramètres soient convertis en chaînes.

Ceci est un exemple de la façon dont un test unitaire nous aide en termes de conception de notre code : en essayant de tester le code, nous en avons trouvé une meilleure version. Nous pouvons aller encore plus loin et dire que ce test n'est pas assez bon, car le test unitaire surcharge un collaborateur interne du client wrapper dans la deuxième ligne. Pour tenter de résoudre ce problème, nous pourrions dire que le client réel doit être fourni par un paramètre (en utilisant l'injection de dépendances), au lieu de le créer dans sa méthode d'initialisation. Et encore une fois, le test unitaire nous a fait penser à une meilleure implémentation.

Le corollaire de l'exemple précédent devrait être que la testabilité d'un morceau de code parle aussi de sa qualité. En d'autres termes, si le code est difficile à tester ou si ses tests sont compliqués, alors il doit probablement être amélioré.

## Dénir les limites de ce qu'il faut tester

Tester demande des efforts. Et si nous ne faisons pas attention au moment de décider quoi tester, nous ne finirons jamais les tests, gaspillant ainsi beaucoup d'efforts sans parvenir à grand-chose.


Nous devons étendre les tests aux limites de notre code. Si nous ne le faisons pas, nous devrons également tester les dépendances (bibliothèques ou modules externes/tiers) dans notre code, puis leurs dépendances respectives, et ainsi de suite dans un voyage sans fin. Ce n'est pas notre responsabilité de tester les dépendances, nous pouvons donc supposer que ces projets ont leurs propres tests. Il suffirait de tester que les appels corrects aux dépendances externes sont effectués avec les paramètres corrects (et cela pourrait même être une utilisation acceptable du patch), mais nous ne devrions pas faire plus d'efforts que cela.

Il s'agit d'un autre cas où une bonne conception logicielle porte ses fruits. Si nous avons été prudents dans notre conception et défini clairement les limites de notre système (c'est-à-dire que nous avons conçu vers des interfaces, au lieu d'implémentations concrètes qui changeront, inversant ainsi les dépendances sur les composants externes pour réduire le couplage temporel), alors il sera beaucoup plus facile de se moquer de ces interfaces lors de l'écriture des tests unitaires.

Dans un bon test unitaire, nous voulons corriger les limites de notre système et nous concentrer sur la fonctionnalité de base à exercer. Nous ne testons pas les bibliothèques externes (outils tiers installés via pip, par exemple), mais nous vérifions plutôt qu'elles sont correctement appelées. Lorsque nous explorerons les objets fictifs plus loin dans ce guide, nous passerons en revue les techniques et les outils permettant d'effectuer ces types d'assertion.

## Des outils pour tester

Il existe de nombreux outils que nous pouvons utiliser pour écrire nos tests unitaires, tous avec des avantages et des inconvénients et servant des objectifs différents. Je vais présenter les deux bibliothèques les plus couramment utilisées pour les tests unitaires en Python. Ils couvrent la plupart (sinon tous) des cas d'utilisation, et ils sont très populaires, donc savoir comment les utiliser est pratique.

En plus des frameworks de test et des bibliothèques d'exécution de tests, il est souvent courant de trouver des projets qui congurent la couverture de code, qu'ils utilisent comme métriques de qualité. Étant donné que la couverture (lorsqu'elle est utilisée comme métrique) est trompeuse, après avoir vu comment créer des tests unitaires, nous expliquerons pourquoi elle ne doit pas être prise à la légère.

La section suivante commence par présenter les principales bibliothèques que nous allons utiliser dans ce chapitre pour les tests unitaires


## Frameworks et bibliothèques pour les tests unitaires

Dans cette section, nous allons discuter de deux frameworks pour écrire et exécuter des tests unitaires. Le premier, unittest, est disponible dans la bibliothèque standard de Python, tandis que le second, pytest, doit être installé en externe via pip

* unittest: https://docs.python.org/3/library/unittest.html
*   pytest: https://docs.pytest.org/en/latest

Lorsqu'il s'agit de couvrir les scénarios de test pour notre code, unittest seul suffira probablement, car il a beaucoup d'aides. Cependant, pour les systèmes plus complexes sur lesquels nous avons de multiples dépendances, des connexions à des systèmes externes et probablement le besoin de patcher des objets, de définir des montages et de paramétrer des cas de test, alors pytest ressemble à une option plus complète.

Nous utiliserons un petit programme comme exemple pour vous montrer comment il pourrait être testé en utilisant les deux options, ce qui, en fin de compte, nous aidera à avoir une meilleure idée de la façon dont les deux se comparent.

L'exemple démontrant les outils de test est une version simplifiée d'un outil de contrôle de version qui prend en charge les revues de code dans les demandes de fusion. Nous commencerons par les critères suivants.

* Une demande de fusion est rejetée si au moins une personne n'est pas d'accord avec les modifications

* Si personne n'est en désaccord et que la demande de fusion est bonne pour au moins deux autres développeurs, elle est approuvée

* Dans tous les autres cas, son statut est en attente

Et voici à quoi pourrait ressembler le code :

In [ ]:
from enum import Enum


class MergeRequestStatus(Enum):
    APPROVED = "approved"
    REJECTED = "rejected"
    PENDING = "pending"


class MergeRequest:
    """An entity abstracting a merge request."""

    def __init__(self):
        self._context = {"upvotes": set(), "downvotes": set()}

    @property
    def status(self):
        if self._context["downvotes"]:
            return MergeRequestStatus.REJECTED
        elif len(self._context["upvotes"]) >= 2:
            return MergeRequestStatus.APPROVED
        return MergeRequestStatus.PENDING

    def upvote(self, by_user):
        self._context["downvotes"].discard(by_user)
        self._context["upvotes"].add(by_user)

    def downvote(self, by_user):
        self._context["upvotes"].discard(by_user)
        self._context["downvotes"].add(by_user)

En utilisant ce code comme base, voyons comment il peut être testé unitairement en utilisant les deux bibliothèques présentées dans ce guide. L'idée n'est pas seulement d'apprendre à utiliser chaque bibliothèque, mais aussi d'identifier certaines différences

## unittest

Le module unittest est une excellente option pour commencer à écrire des tests unitaires car il fournit une API riche pour écrire toutes sortes de conditions de test, et puisqu'il est disponible dans la bibliothèque standard, il est assez polyvalent et pratique.

Le module unittest est basé sur les concepts de JUnit (de Java), qui, à leur tour, est également basé sur les idées originales de tests unitaires qui viennent de Smalltalk (c'est peut-être la raison de la convention de nommage des méthodes sur ce module ), il est donc de nature orientée objet. Pour cette raison, les tests sont écrits via des classes, où les contrôles sont vérifiés par des méthodes, et il est courant de regrouper les tests par scénarios dans des classes.

Pour commencer à écrire des tests unitaires, nous devons créer une classe de test qui hérite de unittest.TestCase, et dénir les conditions sur lesquelles nous voulons mettre l'accent sur ses méthodes. Ces méthodes doivent commencer par test_* et peuvent utiliser en interne n'importe laquelle des méthodes héritées de unittest.TestCase pour vérifier les conditions qui doivent être vraies

Voici quelques exemples de conditions que nous pourrions vouloir vérifier pour notre cas :

In [ ]:
class TestMergeRequestStatus(unittest.TestCase):    
  def test_simple_rejected(self):        
    merge_request = MergeRequest()        
    merge_request.downvote("maintainer")        
    self.assertEqual(merge_request.status, MergeRequestStatus.REJECTED)    
    
    
  def test_just_created_is_pending(self):        
    self.assertEqual(MergeRequest().status, MergeRequestStatus.PENDING)    
  
  
  def test_pending_awaiting_review(self):        
    
    merge_request = MergeRequest()        
    merge_request.upvote("core-dev")        
    self.assertEqual(merge_request.status, MergeRequestStatus.PENDING)    
    
  def test_approved(self):        
    merge_request = MergeRequest()        
    merge_request.upvote("dev1")        
    merge_request.upvote("dev2")        
    self.assertEqual(merge_request.status, MergeRequestStatus.APPROVED)

L'API pour les tests unitaires fournit de nombreuses méthodes utiles pour la comparaison, la plus courante étant assertEqual(<actual>, <expected>[, message]), qui peut être utilisée pour comparer le résultat de l'opération avec la valeur que nous attendions, éventuellement à l'aide d'un message qui s'affichera en cas d'erreur.

J'ai nommé les paramètres en utilisant l'ordre (<réel>, <attendu>), car c'est l'ordre que j'ai trouvé la plupart du temps dans mon expérience. Même si je pense que c'est la forme la plus courante (en tant que convention) à utiliser en Python, il n'y a pas de recommandations ou de directives à ce sujet. En fait, certains projets (comme gRPC) utilisent la forme inverse (<expected>, <actual>), et c'est en fait une convention dans d'autres langages (par exemple, Java et Kotlin). La clé est d'être cohérent et de respecter la forme qui a déjà été utilisée dans votre projet.

Une autre méthode de test utile nous permet de vérifier si une certaine exception a été levée ou non (assertRaises)

Lorsque quelque chose d'exceptionnel se produit, nous levons une exception dans notre code pour empêcher un traitement ultérieur sous de mauvaises hypothèses, et également pour informer l'appelant que quelque chose ne va pas avec l'appel tel qu'il a été effectué. C'est la partie de la logique qui doit être testée, et c'est à cela que sert cette méthode.

Imaginez que nous étendions maintenant notre logique un peu plus loin pour permettre aux utilisateurs de fermer leurs demandes de fusion, et une fois que cela se produit, nous ne voulons plus qu'il y ait de votes (cela n'aurait aucun sens d'évaluer une demande de fusion une fois c'était déjà fermé). Pour éviter que cela ne se produise, nous étendons notre code et nous levons une exception sur l'événement malheureux où quelqu'un essaie de voter sur une demande de fusion fermée.

Après avoir ajouté deux nouveaux statuts (OPEN et CLOSED), et une nouvelle méthode close(), nous modifions les précédentes méthodes de vote pour gérer cette vérification en premie

In [ ]:
from enum import Enum

class MergeRequestException(Exception):
    """Something went wrong with the merge request."""


class MergeRequestStatus(Enum):
    APPROVED = "approved"
    REJECTED = "rejected"
    PENDING = "pending"
    OPEN = "open"
    CLOSED = "closed"

class MergeRequest:
    def __init__(self):
        self._context = {"upvotes": set(), "downvotes": set()}
        self._status = MergeRequestStatus.OPEN

    def close(self):
        self._status = MergeRequestStatus.CLOSED

    @property
    def status(self):
        if self._status == MergeRequestStatus.CLOSED:
            return self._status

        return AcceptanceThreshold(self._context).status()

    def _cannot_vote_if_closed(self):
        if self._status == MergeRequestStatus.CLOSED:
            raise MergeRequestException("can't vote on a closed merge request")

    def upvote(self, by_user):
        self._cannot_vote_if_closed()

        self._context["downvotes"].discard(by_user)
        self._context["upvotes"].add(by_user)

    def downvote(self, by_user):
        self._cannot_vote_if_closed()

        self._context["upvotes"].discard(by_user)
        self._context["downvotes"].add(by_user)

Maintenant, nous voulons vérifier que cette validation fonctionne bien. Pour cela, nous allons utiliser les méthodes asssertRaises et assertRaisesRegex

In [ ]:
def test_cannot_upvote_on_closed_merge_request(self):
        self.merge_request.close()
        self.assertRaises(
            MergeRequestException, self.merge_request.upvote, "dev1"
        )

def test_cannot_downvote_on_closed_merge_request(self):
    self.merge_request.close()
    self.assertRaisesRegex(
        MergeRequestException,
        "can't vote on a closed merge request",
        self.merge_request.downvote,
        "dev1",
    )

Le premier s'attendra à ce que l'exception fournie soit levée lors de l'appel de l'appelable dans le deuxième argument, avec les arguments (*args et **kwargs) sur le reste de la fonction, et si ce n'est pas le cas, il échouera, en disant que l'exception qui devait être levée ne l'était pas. Ce dernier fait de même, mais il vérifie également que l'exception qui a été levée contient le message correspondant à l'expression régulière qui a été fournie en paramètre. Même si l'exception est levée, mais avec un message différent (ne correspondant pas à l'expression régulière), le test échouera.


Notez comment ces méthodes peuvent également être utilisées comme gestionnaires de contexte. Dans sa première forme (celle utilisée dans les exemples précédents), la méthode prend l'exception, puis l'appelable, et enfin la liste des arguments à utiliser dans cet appelable). Mais nous pourrions également passer l'exception en tant que paramètre de la méthode, l'utiliser comme gestionnaire de contexte et évaluer notre code à l'intérieur du bloc de ce gestionnaire de contexte, dans ce format

    with self.assertRaises(MyException):   
      test_logic()

Cette seconde forme est généralement plus utile (et parfois, la seule option) ; par exemple, si la logique que nous devons tester ne peut pas être exprimée comme un seul appelable.

Dans certains cas, vous remarquerez que nous devons exécuter le même scénario de test, mais avec des données différentes. Au lieu de répéter et de générer des tests dupliqués, nous pouvons en construire un seul et exercer sa condition avec différentes valeurs. C'est ce qu'on appelle des tests paramétrés, et nous allons commencer à les explorer dans la section suivante. Plus tard, nous revisiterons les tests paramétrés avec pytest

## Tests paramétrés

Maintenant, nous aimerions tester le fonctionnement du seuil d'acceptation de la demande de fusion, simplement en fournissant des exemples de données de ce à quoi ressemble le contexte sans avoir besoin de l'intégralité de l'objet MergeRequest. Nous voulons tester la partie de la propriété status qui se trouve après la ligne qui vérifie si elle est fermée, mais indépendamment.

Le meilleur moyen d'y parvenir est de séparer ce composant dans une autre classe, d'utiliser la composition, puis de tester cette nouvelle abstraction avec sa propre suite de tests



In [ ]:
class AcceptanceThreshold:
    def __init__(self, merge_request_context: dict) -> None:
        self._context = merge_request_context

    def status(self):
        if self._context["downvotes"]:
            return MergeRequestStatus.REJECTED
        elif len(self._context["upvotes"]) >= 2:
            return MergeRequestStatus.APPROVED
        return MergeRequestStatus.PENDING


class MergeRequest:
    def __init__(self):
        self._context = {"upvotes": set(), "downvotes": set()}
        self._status = MergeRequestStatus.OPEN

    def close(self):
        self._status = MergeRequestStatus.CLOSED

    @property
    def status(self):
        if self._status == MergeRequestStatus.CLOSED:
            return self._status

        return AcceptanceThreshold(self._context).status()

    def _cannot_vote_if_closed(self):
        if self._status == MergeRequestStatus.CLOSED:
            raise MergeRequestException("can't vote on a closed merge request")

    def upvote(self, by_user):
        self._cannot_vote_if_closed()

        self._context["downvotes"].discard(by_user)
        self._context["upvotes"].add(by_user)

    def downvote(self, by_user):
        self._cannot_vote_if_closed()

        self._context["upvotes"].discard(by_user)
        self._context["downvotes"].add(by_user)

Avec ces changements, nous pouvons relancer les tests et vérifier qu'ils réussissent, ce qui signifie que ce petit refactor n'a rien cassé de la fonctionnalité actuelle (les tests unitaires assurent la régression). Avec cela, nous pouvons poursuivre notre objectif d'écrire des tests spécifiques à la nouvelle classe

In [ ]:
from unittest import TestCase, main

class TestAcceptanceThreshold(TestCase):
    def setUp(self):
        self.fixture_data = (
            (
                {"downvotes": set(), "upvotes": set()},
                MergeRequestExtendedStatus.PENDING,
            ),
            (
                {"downvotes": set(), "upvotes": {"dev1"}},
                MergeRequestExtendedStatus.PENDING,
            ),
            (
                {"downvotes": "dev1", "upvotes": set()},
                MergeRequestExtendedStatus.REJECTED,
            ),
            (
                {"downvotes": set(), "upvotes": {"dev1", "dev2"}},
                MergeRequestExtendedStatus.APPROVED,
            ),
        )

    def test_status_resolution(self):
        for context, expected in self.fixture_data:
            with self.subTest(context=context):
                status = AcceptanceThreshold(context).status()
                self.assertEqual(status, expected)




Ici, dans la méthode setUp(), nous définissons la structure de données à utiliser tout au long des tests. Dans ce cas, ce n'est pas vraiment nécessaire, car nous aurions pu le mettre directement sur la méthode, mais si nous prévoyons d'exécuter du code avant l'exécution d'un test, c'est l'endroit pour l'écrire, car cette méthode est appelée une fois avant chaque le test est exécuté.

Dans ce cas particulier, nous aurions pu définir ce tuple comme un attribut de classe, car il s'agit d'une valeur constante (statique). Si nous avions besoin d'exécuter du code et d'effectuer des calculs (comme la construction d'objets ou l'utilisation d'une usine), la méthode setUp() est notre seule alternative.

En écrivant cette nouvelle version du code, les paramètres sous le code testé sont plus clairs et plus compacts


Pour simuler que nous exécutons tous les paramètres, le test itère sur toutes les données et exerce le code avec chaque instance. Une aide intéressante ici est l'utilisation de subTest, que nous utilisons dans ce cas pour marquer la condition de test appelée. Si l'une de ces itérations échouait, unittest le rapporterait avec la valeur correspondante des variables qui ont été passées au subTest (dans ce cas, il a été nommé context, mais n'importe quelle série d'arguments de mot-clé fonctionnerait de la même manière).

L'idée derrière les tests paramétrés est d'exécuter la même condition de test sur différents ensembles de données. L'idée est que vous identifiiez d'abord les classes d'équivalence des données à tester, puis que vous choisissiez le représentant de la valeur de chaque classe (plus de détails à ce sujet plus loin dans le guide). Ensuite, vous souhaitez savoir pour quelle classe d'équivalence votre test a échoué, et le contexte fourni par le gestionnaire de contexte subTest est utile dans ce cas

## pytest

Pytest est un excellent framework de test et peut être installé via pip install pytest. Une différence par rapport à unittest est que, bien qu'il soit toujours possible de classer les scénarios de test dans des classes et de créer des modèles orientés objet de nos tests, ce n'est pas réellement obligatoire, et il est possible d'écrire des tests unitaires avec moins de passe-partout en vérifiant simplement les conditions nous voulons vérifier dans des fonctions simples avec l'instruction assert.

Par défaut, faire des comparaisons avec une instruction assert sera suffisant pour que pytest identifie un test unitaire et rapporte son résultat en conséquence. Des utilisations plus poussées, comme celles vues dans la section précédente, sont également possibles, mais elles nécessitent l'utilisation de fonctions spéciques du package.

Une fonctionnalité intéressante est que la commande pytests exécutera tous les tests qu'elle peut découvrir, même s'ils ont été écrits avec unittest. Cette compatibilité facilite le passage progressif d'unittest à pytest

## Cas de test de base avec pytest

Les conditions que nous avons testées dans la section précédente peuvent être réécrites en fonctions simples avec pytes.

Voici quelques exemples avec des assertions simples

In [ ]:
def test_simple_rejected():
    merge_request = MergeRequest()
    merge_request.downvote("maintainer")
    assert merge_request.status == MergeRequestStatus.REJECTED


def test_just_created_is_pending():
    assert MergeRequest().status == MergeRequestStatus.PENDING


def test_pending_awaiting_review():
    merge_request = MergeRequest()
    merge_request.upvote("core-dev")
    assert merge_request.status == MergeRequestStatus.PENDING

Les comparaisons d'égalité booléennes ne nécessitent pas plus qu'une simple instruction assert, alors que d'autres types de vérifications, telles que celles des exceptions, nécessitent l'utilisation de certaines fonctions

In [ ]:
def test_invalid_types():
    merge_request = MergeRequest()
    pytest.raises(TypeError, merge_request.upvote, {"invalid-object"})


def test_cannot_vote_on_closed_merge_request():
    merge_request = MergeRequest()
    merge_request.close()
    pytest.raises(MergeRequestException, merge_request.upvote, "dev1")
    with pytest.raises(
        MergeRequestException,
        match="can't vote on a closed merge request",
    ):
        merge_request.downvote("dev1")

Dans ce cas, pytest.raises est l'équivalent de unittest.TestCase.assertRaises, et il accepte également qu'il soit appelé à la fois comme méthode et comme gestionnaire de contexte. Si nous voulons vérifier le message de l'exception, au lieu d'une méthode différente (comme assertRaisesRegex), la même fonction doit être utilisée, mais en tant que gestionnaire de contexte, et en fournissant le paramètre match avec l'expression que nous aimerions identifier.

pytest encapsulera également l'exception d'origine dans une exception personnalisée à laquelle on peut s'attendre (en vérifiant certains de ses attributs, tels que .value, par exemple) au cas où nous voudrions vérifier plus de conditions, mais cette utilisation de la fonction couvre le vaste majorité des cas

## Tests paramétrés

Exécuter des tests paramétrés avec pytest est mieux, non seulement parce qu'il fournit une API plus propre, mais aussi parce que chaque combinaison du test avec ses paramètres génère un nouveau cas de test (une nouvelle fonction).

Pour travailler avec cela, nous devons utiliser le décorateur pytest.mark.parametrize sur notre test. Le premier paramètre du décorateur est une chaîne indiquant les noms des paramètres à passer à la fonction de test, et le second doit être itérable avec les valeurs respectives de ces paramètres.

Remarquez comment le corps de la fonction de test est réduit à une ligne (après avoir supprimé la boucle for interne et son gestionnaire de contexte imbriqué), et les données de chaque cas de test sont correctement isolées du corps de la fonction, ce qui facilite l'extension et maintenir

In [ ]:
@pytest.mark.parametrize(
    "context,expected_status",
    (
        ({"downvotes": set(), "upvotes": set()}, MergeRequestStatus.PENDING),
        (
            {"downvotes": set(), "upvotes": {"dev1"}},
            MergeRequestStatus.PENDING,
        ),
        ({"downvotes": "dev1", "upvotes": set()}, MergeRequestStatus.REJECTED),
        (
            {"downvotes": set(), "upvotes": {"dev1", "dev2"}},
            MergeRequestStatus.APPROVED,
        ),
    ),
)
def test_acceptance_threshold_status_resolution(context, expected_status):
    assert AcceptanceThreshold(context).status() == expected_status

Une recommandation importante lors de l'utilisation de la paramétrisation est que chaque paramètre (chaque itération) doit correspondre à un seul scénario de test. Cela signifie que vous ne devez pas mélanger différentes conditions de test dans le même paramètre. Si vous devez tester la combinaison de différents paramètres, utilisez différentes paramétrisations empilées. L'empilement de ce décorateur créera autant de conditions de test que le produit cartésien de toutes les valeurs dans les décorateurs

In [ ]:
@pytest.mark.parametrize("x", (1, 2))
@pytest.mark.parametrize("y", ("a", "b"))
def my_test(x, y):   …

Fonctionnera pour les valeurs (x = 1, y = a), (x = 1, y = b), (x = 2, y = a) et (x = 2, y = b)


C'est une meilleure approche car chaque test est plus petit et chaque paramétrisation plus spécifique (cohésive). Il vous permettra de souligner le code avec l'explosion de toutes les combinaisons possibles de manière plus simple.


Les paramètres de données fonctionnent bien lorsque vous avez les données que vous devez tester, ou que vous savez comment les construire facilement, mais dans certains cas, vous avez besoin d'objets spécifiques à construire pour un test, ou vous vous retrouvez à écrire ou à construire le les mêmes objets à plusieurs reprises. Pour nous aider, nous pouvons utiliser des fixtures, comme nous le verrons dans la section suivante

## Fixtures

L'un des avantages de pytest est qu'il facilite la création de fonctionnalités réutilisables afin que nous puissions alimenter nos tests avec des données ou des objets à tester plus efficacement et sans répétition

Par exemple, nous pourrions vouloir créer un objet MergeRequest dans un état particulier et utiliser cet objet dans plusieurs tests. Nous définissons notre objet comme une fixture en créant une fonction et en appliquant le décorateur @pytest.fixture. Les tests qui veulent utiliser cette fixture devront avoir un paramètre avec le même nom que la fonction qui est dénie, et pytest s'assurera qu'il est fourni

In [ ]:
import pytest

@pytest.fixture
def rejected_mr():
    merge_request = MergeRequest()

    merge_request.downvote("dev1")
    merge_request.upvote("dev2")
    merge_request.upvote("dev3")
    merge_request.downvote("dev4")

    return merge_request

def test_simple_rejected(rejected_mr):
    assert rejected_mr.status == MergeRequestStatus.REJECTED


def test_rejected_with_approvals(rejected_mr):
    rejected_mr.upvote("dev2")
    rejected_mr.upvote("dev3")
    assert rejected_mr.status == MergeRequestStatus.REJECTED


def test_rejected_to_pending(rejected_mr):
    rejected_mr.upvote("dev1")
    assert rejected_mr.status == MergeRequestStatus.PENDING


def test_rejected_to_approved(rejected_mr):
    rejected_mr.upvote("dev1")
    rejected_mr.upvote("dev2")
    assert rejected_mr.status == MergeRequestStatus.APPROVED


N'oubliez pas que les tests affectent également le code principal, donc les principes du code propre s'appliquent également à eux. Dans ce cas, le principe Don't Repeat Yourself (DRY) que nous avons exploré dans les guides précédents apparaît à nouveau, et nous pouvons y parvenir à l'aide de pytest fixtures.

En plus de créer plusieurs objets ou d'exposer des données qui seront utilisées tout au long de la suite de tests, il est également possible de les utiliser pour configurer certaines conditions, par exemple, pour patcher globalement certaines fonctions que nous ne voulons pas être appelées, ou quand nous voulons patcher les objets à utiliser à la place

## Couverture de code

Les exécuteurs de tests prennent en charge les plugins de couverture (à installer via pip) qui fournissent des informations utiles sur les lignes du code qui ont été exécutées lors de l'exécution des tests. Ces informations sont d'une grande aide pour savoir quelles parties du code doivent être couvertes par des tests, ainsi que pour identifier les améliorations à apporter (à la fois dans le code de production et dans les tests). Ce que je veux dire par là, c'est que détecter les lignes de notre code de production qui sont découvertes nous obligera à écrire un test pour cette partie du code (car rappelez-vous que le code qui n'a pas de tests doit être considéré comme cassé). Dans cette tentative de couvrir le code, plusieurs choses peuvent se produire :

*  Nous pourrions réaliser que nous avons complètement raté un scénario de test.

*  Nous allons essayer de proposer plus de tests unitaires ou de tests unitaires qui couvrent plus de lignes de code.

* Nous essaierons de simplifier notre code de production, en supprimant les redondances et en le rendant plus compact, ce qui signifie qu'il est plus facile à couvrir

* Nous pourrions même réaliser que les lignes de code que nous essayons de couvrir sont inaccessibles (il y a peut-être eu une erreur dans la logique) et peuvent être supprimées en toute sécurité


Gardez à l'esprit que même s'il s'agit de points positifs, la couverture ne doit jamais être une cible, mais seulement une mesure. Cela signifie qu'essayer d'atteindre une couverture élevée, juste pour atteindre 100 %, ne sera ni productif ni efficace. Nous devons comprendre la couverture du code comme une unité pour identifier les parties évidentes du code qui doivent être testées et voir comment nous pouvons l'améliorer. On peut cependant fixer un seuil minimum de disons 80% (une valeur généralement acceptée) comme niveau minimum de couverture souhaitée pour savoir que le projet a un nombre raisonnable de tests.


De plus, penser qu'un degré élevé de couverture de code est le signe d'une base de code saine est également dangereux : gardez à l'esprit que la plupart des outils de couverture rapporteront sur les lignes de production de code qui ont été exécutées. Qu'une ligne ait été appelée ne signifie pas qu'elle a été correctement testée (seulement qu'elle a fonctionné). Une seule instruction peut encapsuler plusieurs conditions logiques, chacune devant être testée séparément.


    Ne vous laissez pas égarer par un degré élevé de couverture de code et 
    continuez à réfléchir aux moyens de tester le code, y compris les lignes déjà couvertes


L'une des bibliothèques les plus utilisées pour cela est la couverture (https://pypi.org/project/coverage/). Nous verrons comment configurer cet outil dans la section suivante

 

## Mise en place d'une couverture de repos

Dans le cas de pytest, nous pouvons installer le package pytest-cov. Une fois installé, lorsque les tests sont exécutés, nous devons dire au pytest runner que pytest-cov s'exécutera également, et quel package (ou packages) doit être couvert (entre autres paramètres et configurations)

Ce package prend en charge plusieurs configurations, y compris différents types de formats de sortie, et il est facile de l'intégrer à n'importe quel outil CI, mais parmi toutes ces fonctionnalités, une option fortement recommandée est de définir le drapeau qui nous indiquera quelles lignes n'ont pas n'a pas encore été couvert par des tests, car c'est ce qui va nous aider à diagnostiquer notre code et nous permettre de commencer à écrire plus de tests.

Pour vous montrer un exemple de ce à quoi cela ressemblerait, utilisez la commande suivante


    PYTHONPATH=src pytest \ 
       --cov-report term-missing \
      --cov=coverage_1 \    
      tests/test_coverage_1.py

Cela produira une sortie similaire à la suivante



      test_coverage_1.py ................ [100%]
      
      
      ----------- coverage: platform linux, python 3.6.5-final-0 -----------
      Name         Stmts Miss Cover Missing
      ---------------------------------------------
      coverage_1.py 39      1  97%    44


Ici, cela nous dit qu'il y a une ligne qui n'a pas de tests unitaires afin que nous puissions jeter un œil et voir comment écrire un test unitaire pour cela. Il s'agit d'un scénario courant où nous réalisons que pour couvrir ces lignes manquantes, nous devons refactoriser le code en créant des méthodes plus petites. En conséquence, notre code sera bien meilleur, comme dans l'exemple que nous avons vu au début de ce guide.

Le problème réside dans la situation inverse : pouvons-nous faire confiance à la couverture élevée ? Cela signifie-t-il que notre code est correct ? Malheureusement, avoir une bonne couverture de test est une condition nécessaire mais insufsante pour un code propre. Ne pas avoir de tests pour des parties du code est clairement quelque chose de mauvais. Avoir des tests est en fait très bien, mais nous ne pouvons le dire que pour les tests qui existent. Cependant, nous ne savons pas grand-chose sur les tests qui nous manquent, et nous pouvons manquer de nombreuses conditions même lorsque la couverture du code est élevée.

Ce sont quelques-unes des mises en garde de la couverture des tests, que nous mentionnerons dans la section suivante

## Mises en garde de la couverture des tests

Python est interprété et, à un très haut niveau, les outils de couverture en profitent pour identifier les lignes qui ont été interprétées (exécutées) lors de l'exécution des tests. Il le rapportera ensuite à la fin. Le fait qu'une ligne ait été interprétée ne signifie pas qu'elle a été correctement testée, et c'est pourquoi il faut faire attention à la lecture du rapport de couverture final et se fier à ce qu'il dit.

c'est en fait vrai pour n'importe quel language. Le fait qu'une ligne ait été exercée ne signifie pas du tout qu'elle a été soulignée avec toutes ses combinaisons possibles. Le fait que toutes les branches s'exécutent avec succès avec les données fournies signifie uniquement que le code prend en charge cette combinaison, mais cela ne nous dit rien sur les autres combinaisons possibles de paramètres qui feraient planter le programme (test flou)


    Utilisez la couverture comme un outil pour trouver les angles morts dans le
    code, mais pas comme une métrique ou un objectif cible.

Pour illustrer cela avec un exemple simple, considérons le code suivant:


In [ ]:
def my_function(number: int):    
  return "even" if number % 2 == 0 else "odd"

Maintenant, disons que nous écrivons le test suivant pour cela




In [ ]:
@pytest.mark.parametrize("number,expected", [(2, "even")])
def test_my_function(number, expected):
    assert my_function(number) == expected

Si nous exécutons les tests avec couverture, le rapport nous donnera une couverture clignotante à 100 %. Inutile de dire qu'il nous manque un test pour la moitié des conditions de l'instruction unique qui s'est exécutée. Encore plus troublant est le fait que puisque la clause else de l'instruction ne s'est pas exécutée, nous ne savons pas de quelle manière notre code pourrait se briser (pour rendre cet exemple encore plus exagéré, imaginez qu'il y avait une instruction incorrecte, telle que 1 /0 au lieu de la chaîne "impair", ou qu'il y a un appel de fonction).


Sans doute, nous pourrions aller plus loin et penser que ce n'est que le "chemin heureux" parce que nous fournissons de bonnes valeurs à la fonction. Mais qu'en est-il des types incorrects ? Comment la fonction doit-elle se défendre contre cela ?


Comme vous le voyez, même une seule déclaration d'apparence innocente peut déclencher de nombreuses questions et conditions de test auxquelles nous devons être préparés.

C'est une bonne idée de vérifier dans quelle mesure notre code est couvert, et même de configurer des seuils de couverture de code dans le cadre de la construction de CI, mais nous devons garder à l'esprit qu'il ne s'agit que d'un autre outil pour nous. Et tout comme les outils précédents que nous avons explorés (linters, code checkers, formatters, and suchlike)), il n'est utile que dans le contexte de davantage d'outils et d'un bon environnement préparé pour une base de code propre.

Un autre outil qui nous aidera dans nos efforts de test est l'utilisation d'objets fictifs. Nous les explorons dans la section suivante

## Mock objects

Il y a des cas où notre code n'est pas la seule chose qui sera présente dans le cadre de nos tests. Après tout, les systèmes que nous concevons et construisons doivent faire quelque chose de réel, ce qui signifie généralement se connecter à des services externes (bases de données, services de stockage, API externes, services cloud, etc.). Parce qu'ils doivent avoir ces effets secondaires, ils sont inévitables. Autant nous faisons abstraction de notre code, programmons vers des interfaces et isolons le code des facteurs externes pour minimiser les effets secondaires, ils seront présents dans nos tests, et nous avons besoin d'un moyen efficace de gérer cela.

Les objets mock sont l'une des meilleures tactiques utilisées pour protéger nos tests unitaires contre les effets secondaires indésirables (comme vu plus haut dans ce guide). Notre code peut avoir besoin d'effectuer une requête HTTP ou d'envoyer un e-mail de notification, mais nous ne voulons certainement pas que cela se produise dans nos tests unitaires. Les tests unitaires doivent cibler la logique de notre code et s'exécuter rapidement, car nous voulons les exécuter assez souvent, ce qui signifie que nous ne pouvons pas nous permettre de latence. Par conséquent, les tests unitaires réels n'utilisent aucun service réel - ils ne se connectent à aucune base de données, ils n'émettent pas de requêtes HTTP et, fondamentalement, ils ne font rien d'autre que d'exercer la logique du code de production.

Nous avons besoin de tests qui font de telles choses, mais ce ne sont pas des unités. Les tests d'intégration sont censés tester les fonctionnalités avec une perspective plus large, imitant presque le comportement d'un utilisateur. Mais ils ne sont pas rapides. Parce qu'ils se connectent à des systèmes et services externes, ils prennent plus de temps et sont plus coûteux à exécuter. En général, nous aimerions avoir beaucoup de tests unitaires qui s'exécutent rapidement afin de les exécuter tout le temps et que les tests d'intégration s'exécutent moins souvent (par exemple, sur toute nouvelle demande de fusion).

## Un avertissement juste sur les patching et les mocks

J'ai dit auparavant que les tests unitaires nous aident à écrire un meilleur code, car dès que nous commencerons à réfléchir à la façon de tester notre code, nous réaliserons comment il peut être amélioré pour le rendre testable. Et généralement, à mesure que le code devient plus testable, il devient plus propre (plus cohérent, granulaire, divisé en composants plus petits, etc.)

Un autre gain intéressant est que les tests nous aideront à remarquer les odeurs de code dans les parties où nous pensions que notre code était correct. L'un des principaux avertissements que notre code a des odeurs de code est de savoir si nous nous trouvons en train d'essayer de patcher (ou mock) de beaucoup de choses différentes juste pour couvrir un cas de test simple

Le module unittest fournit un outil pour patcher nos objets sur unittest.mock.patch.

Le patch signifie que le code d'origine (donné par une chaîne indiquant son emplacement au moment de l'importation) sera remplacé par autre chose que son code d'origine. Si aucun objet de remplacement n'est fourni, la valeur par défaut est un objet mock standard qui acceptera simplement tous les appels de méthode ou attributs.

La fonction de patch remplace le code au moment de l'exécution et a l'inconvénient de perdre le contact avec le code d'origine qui était là en premier lieu, ce qui rend nos tests un peu moins profonds. Il comporte également des considérations de performances en raison de la surcharge qui impose de modifier les objets dans l'interpréteur au moment de l'exécution, et c'est quelque chose qui pourrait nécessiter des modifications futures si nous refactorisons notre code et déplaçons les choses (parce que les chaînes déclarées dans la fonction de correctif ne seront plus valides )

L'utilisation de patchs de singe ou de simulations dans nos tests peut être acceptable, et en soi, cela ne représente pas un problème. D'un autre côté, les abus dans le patching des singes sont en effet un drapeau rouge nous indiquant qu'il faut améliorer quelque chose dans notre cabillaud.

Par exemple, de la même manière que rencontrer des difficultés lors du test d'une fonction pourrait nous donner l'idée que cette fonction est probablement trop grande et devrait être décomposée en plus petits morceaux, en essayant de tester un morceau de code qui nécessite un singe très invasif patch devrait nous dire que le code s'appuie peut-être trop sur des dépendances matérielles, et que l'injection de dépendances devrait être utilisée à la place

## Utiliser des objets Mock

Dans la terminologie des tests unitaires, il existe plusieurs types d'objets qui entrent dans la catégorie nommée test double. Un double de test est un type d'objet qui remplacera un objet réel dans notre suite de tests pour différents types de raisons (nous n'avons peut-être pas besoin du code de production réel, mais juste un objet Mock fonctionnerait, ou peut-être pouvons-nous ne l'utilisez pas car il nécessite un accès à des services ou il a des effets secondaires que nous ne voulons pas dans nos tests unitaires, etc.)

Il existe différents types de doubles de test, tels que des  dummy objects, stubs, spies, or mocks


Les Mocks sont le type d'objet le plus général, et comme ils sont assez flexibles et polyvalents, ils conviennent à tous les cas sans avoir besoin d'entrer dans les détails du reste. C'est pour cette raison que la bibliothèque standard comprend également un objet de ce type, et il est courant dans la plupart des programmes Python. C'est celui que nous allons utiliser ici : unittest.mock.Mock


Un mock est un type d'objet créé selon une spécification (ressemblant généralement à l'objet d'une classe de production) et certaines réponses configurées (c'est-à-dire que nous pouvons dire au mock ce qu'il doit retourner sur certains appels, et quel est son comportement devrait être). L'objet Mock enregistrera alors, dans le cadre de son état interne, comment il a été appelé (avec quels paramètres, combien de fois, etc.), et nous pouvons utiliser ces informations pour vérifier le comportement de notre application à un stade ultérieur.


Dans le cas de Python, l'objet Mock disponible dans la bibliothèque standard fournit une API agréable pour faire toutes sortes d'assertions comportementales, comme vérifier combien de fois la simulation a été appelée, avec quels paramètres, etc

## Types de Mock

La bibliothèque standard fournit des objets Mock et MagicMock dans le module unittest.mock. Le premier est un double de test qui peut être configuré pour renvoyer n'importe quelle valeur et gardera une trace des appels qui lui ont été faits. Ce dernier fait de même, mais il prend également en charge les méthodes magiques. Cela signifie que, si nous avons écrit du code idiomatique qui utilise des méthodes magiques (et que certaines parties du code que nous testons s'appuieront sur cela), il est probable que nous devrons utiliser une instance de MagicMock au lieu d'un simple Mock.

Essayer d'utiliser Mock lorsque notre code doit appeler des méthodes magiques entraînera une erreur. Voir le code suivant pour un exemple de ceci

In [ ]:
from typing import Dict, List


class GitBranch:
    def __init__(self, commits: List[Dict]):
        self._commits = {c["id"]: c for c in commits}

    def __getitem__(self, commit_id):
        return self._commits[commit_id]

    def __len__(self):
        return len(self._commits)


def author_by_id(commit_id, branch):
    return branch[commit_id]["author"]

Nous voulons tester cette fonction ; cependant, un autre test doit appeler la fonction author_by_id. Pour une raison quelconque, puisque nous ne testons pas cette fonction, toute valeur fournie à cette fonction (et renvoyée) sera bonne

In [ ]:
from unittest.mock import MagicMock, Mock


def test_find_commit():
    branch = GitBranch([{"id": "123", "author": "dev1"}])
    assert author_by_id("123", branch) == "dev1"


def test_find_any():
    author = author_by_id("123", Mock()) is not None


Utiliser MagicMock à la place fonctionnera. On peut même configurer la méthode magique de ce type de mock pour retourner quelque chose dont on a besoin afin de contrôler l'exécution de notre test

In [ ]:
def test_find_any():
    mbranch = MagicMock()
    mbranch.__getitem__.return_value = {"author": "test"}
    assert author_by_id("123", mbranch) == "test"

## Un cas d'utilisation pour les doubles de test

Pour voir une éventuelle utilisation de mocks, nous devons ajouter un nouveau composant à notre application qui sera chargé de notifier la demande de fusion de l'état du build. Lorsqu'un build est terminé, cet objet sera appelé avec l'ID de la demande de fusion et le statut du build, et il mettra à jour le statut de la demande de fusion avec ces informations en envoyant une requête HTTP POST à un fixe particulier. point final

In [ ]:
from datetime import datetime

import requests



STATUS_ENDPOINT = "http://localhost:8080/mrstatus"



class BuildStatus:
    """The CI status of a pull request."""

    @staticmethod
    def build_date() -> str:
        return datetime.utcnow().isoformat()

    @classmethod
    def notify(cls, merge_request_id, status):
        build_status = {
            "id": merge_request_id,
            "status": status,
            "built_at": cls.build_date(),
        }
        response = requests.post(STATUS_ENDPOINT, json=build_status)
        response.raise_for_status()
        return response

Cette classe a de nombreux effets secondaires, mais l'un d'entre eux est une dépendance externe importante et difficile à surmonter. Si nous essayons d'écrire un test dessus sans rien modifier, il échouera avec une erreur de connexion dès qu'il essaiera d'effectuer la connexion HTTP.

En tant qu'objectif de test, nous voulons simplement nous assurer que les informations sont correctement composées et que les requêtes de bibliothèque sont appelées avec les paramètres appropriés. Comme il s'agit d'une dépendance externe, nous ne voulons pas tester le module de requêtes ; il suffit de vérifier qu'il s'appelle correctement.

Un autre problème auquel nous serons confrontés en essayant de comparer les données envoyées à la bibliothèque est que la classe calcule l'horodatage actuel, ce qui est impossible à prédire dans un test unitaire. Patcher datetime directement n'est pas possible, car le module est écrit en C. Certaines bibliothèques externes qui peuvent le faire (freezegun, par exemple), mais elles viennent avec une pénalité de performance, et pour cet exemple, ce serait exagéré. Par conséquent, nous choisissons d'envelopper la fonctionnalité que nous voulons dans une méthode statique que nous pourrons patcher

Maintenant que nous avons établi les points à remplacer dans le code, écrivons le test unitaire :

In [ ]:
from unittest import mock


@mock.patch("mock_2.requests")
def test_build_notification_sent(mock_requests):
    build_date = "2018-01-01T00:00:01"
    with mock.patch("mock_2.BuildStatus.build_date", return_value=build_date):
        BuildStatus.notify(123, "OK")

    expected_payload = {"id": 123, "status": "OK", "built_at": build_date}
    mock_requests.post.assert_called_with(
        STATUS_ENDPOINT, json=expected_payload
    )

Tout d'abord, nous utilisons mock.patch comme décorateur pour remplacer le module de requêtes. Le résultat de cette fonction créera un objet mock qui sera passé en paramètre au test (nommé mock_requests dans cet exemple). Ensuite, nous utilisons à nouveau cette fonction, mais cette fois en tant que gestionnaire de contexte pour changer la valeur de retour de la méthode de la classe qui calcule la date de la construction, en remplaçant la valeur par celle que nous contrôlons, que nous utiliserons dans l'assertion.

Une fois que tout cela est en place, nous pouvons appeler la méthode de classe avec certains paramètres, puis nous pouvons utiliser l'objet mock pour vérifier comment il a été appelé. Dans ce cas, nous utilisons la méthode pour voir si request.post a bien été appelé avec les paramètres tels que nous voulions qu'ils soient composés.

C'est une fonctionnalité intéressante des mocks - non seulement ils mettent des limites autour de tous les composants externes (dans ce cas pour empêcher l'envoi de certaines notifications ou l'émission de requêtes HTTP), mais ils fournissent également une API utile pour vérifier les appels et leur paramètres.

Bien que, dans ce cas, nous ayons pu tester le code en mettant en place les objets mock respectifs, il est également vrai que nous avons dû beaucoup patcher proportionnellement au nombre total de lignes de code pour la fonctionnalité principale. Il n'y a pas de règle sur le ratio de code productif pur testé par rapport au nombre de parties de ce code dont nous devons nous moquer, mais certainement, en utilisant le bon sens, nous pouvons voir que, si nous devions patcher pas mal de choses dans le mêmes parties, quelque chose n'est pas clairement abstrait, et cela ressemble à une odeur de code.


Le patching des dépendances externes peut être utilisé en combinaison avec des fixtures pour appliquer certaines configurations globales. Par exemple, c'est généralement une bonne idée d'empêcher tous les tests unitaires d'effectuer des appels HTTP, donc dans le sous-répertoire pour les tests unitaires, nous pouvons ajouter une fixture dans le fichier de configuration de pytest (tests/unit/conftest.py )

In [ ]:
@pytest.fixture(autouse=True)
def no_requests():    
  with patch("requests.post"):        
    yield

Cette fonction sera invoquée automatiquement dans tous les tests unitaires (à cause de autouse=True), et quand c'est le cas, elle patchera la fonction post dans le module de requêtes. Ceci est juste une idée que vous pouvez adapter à vos projets pour ajouter une sécurité supplémentaire et vous assurer que vos tests unitaires sont exempts d'effets secondaires.

Dans la section suivante, nous explorerons comment refactoriser le code pour surmonter ce problème.


## Refactorisation

Refactoriser signifie changer la structure du code en réarrangeant sa représentation interne sans modifier son comportement externe.

Un exemple serait si vous identifiez une classe qui a beaucoup de responsabilités et de très longues méthodes, puis décidez de la modifier en utilisant des méthodes plus petites, en créant de nouveaux collaborateurs internes et en répartissant les responsabilités dans de nouveaux objets plus petits. En faisant cela, vous faites attention à ne pas modifier l'interface d'origine de cette classe, à conserver toutes ses méthodes publiques comme avant et à ne modifier aucune signature. Pour un observateur externe de cette classe, il peut sembler que rien ne s'est passé (mais nous savons le contraire)


Le refactoring est une activité critique dans la maintenance logicielle, pourtant quelque chose qui ne peut pas être fait (du moins pas correctement) sans avoir des tests unitaires. En effet, à chaque modification apportée, nous devons savoir que notre code est toujours correct. Dans un sens, vous pouvez considérer nos tests unitaires comme "l'observateur externe" de notre code, en veillant à ce que le contrat ne se rompe pas.

De temps en temps, nous devons prendre en charge une nouvelle fonctionnalité ou utiliser notre logiciel de manière inattendue. La seule façon de répondre à de telles exigences est d'abord de refactoriser notre code, pour le rendre plus générique ou flexible.

Typiquement, lors de la refactorisation de notre code, nous souhaitons améliorer sa structure et la rendre meilleure, parfois plus générique, plus lisible ou plus flexible. Le défi consiste à atteindre ces objectifs tout en préservant exactement les mêmes fonctionnalités qu'avant les modifications apportées. Cette contrainte de devoir supporter les mêmes fonctionnalités qu'auparavant, mais avec une version différente du code, implique qu'il faut faire des tests de régression sur du code qui a été modié. Le seul moyen rentable d'exécuter des tests de régression est que ces tests soient automatiques. La version la plus économique des tests automatiques est le test unitaire

## Faire évoluer notre code

Dans l'exemple précédent, nous avons pu séparer les effets secondaires de notre code pour le rendre testable en corrigeant les parties du code qui dépendaient de choses que nous ne pouvions pas contrôler lors du test unitaire. C'est une bonne approche car, après tout, la fonction mock.patch est pratique pour ce genre de tâches et remplace les objets auxquels nous lui disons, nous rendant un objet Mock.

L'inconvénient est que nous devons fournir le chemin de l'objet que nous allons simuler, y compris le module, sous forme de chaîne. C'est un peu fragile, car si nous refactorisons notre code (disons que nous renommons le chier ou le déplaçons vers un autre emplacement), tous les endroits avec le patch devront être mis à jour, ou le test échouera

Dans l'exemple, le fait que la méthode notify() dépende directement d'un détail d'implémentation (le module de requêtes) est un problème de conception ; c'est-à-dire qu'il pèse également sur les tests unitaires avec la fragilité susmentionnée qui est implicite.


Nous devons toujours remplacer ces méthodes par des doubles (mocks), mais si nous refactorisons le code, nous pouvons le faire d'une meilleure manière. Séparons ces méthodes en plus petites et, surtout, injectons la dépendance plutôt que de la garder fixe. Le code applique maintenant le principe d'inversion de dépendance, et il s'attend à fonctionner avec quelque chose qui prend en charge une interface (dans cet exemple, une implicite), telle que celle fournie par le module de requêtes

In [ ]:
from datetime import datetime


class BuildStatus:

    endpoint = STATUS_ENDPOINT

    def __init__(self, transport):
        self.transport = transport

    @staticmethod
    def build_date() -> str:
        return datetime.utcnow().isoformat()

    def compose_payload(self, merge_request_id, status) -> dict:
        return {
            "id": merge_request_id,
            "status": status,
            "built_at": self.build_date(),
        }

    def deliver(self, payload):
        response = self.transport.post(self.endpoint, json=payload)
        response.raise_for_status()
        return response

    def notify(self, merge_request_id, status):
        return self.deliver(self.compose_payload(merge_request_id, status))

Nous séparons les méthodes (notez comment notify est maintenant compose Deliver), faisons de compose_payload() une nouvelle méthode (afin que nous puissions remplacer, sans avoir besoin de patcher la classe), et exigeons que la dépendance de transport soit injectée. Maintenant que le transport est une dépendance, il est beaucoup plus facile de changer cet objet pour n'importe quel double que nous voulons.

Il est même possible d'exposer une fixture de cet objet, avec les doubles remplacés au besoin

In [ ]:
from unittest.mock import Mock

import pytest


@pytest.fixture
def build_status():
    bstatus = BuildStatus(Mock())
    bstatus.build_date = Mock(return_value="2018-01-01T00:00:01")
    return bstatus


def test_build_notification_sent(build_status):

    build_status.notify(1234, "OK")

    expected_payload = {
        "id": 1234,
        "status": "OK",
        "built_at": build_status.build_date(),
    }

    build_status.transport.post.assert_called_with(
        build_status.endpoint, json=expected_payload
    )

Comme mentionné dans le premier guide, le but d'avoir du code propre est d'avoir du code maintenable, du code que nous pouvons refactoriser pour qu'il puisse évoluer et s'étendre à plus d'exigences. À cette fin, les tests sont d'une grande aide. Mais comme les tests sont si importants, nous devons également les refactoriser afin qu'ils puissent également conserver leur pertinence et leur utilité au fur et à mesure que le code évolue. C'est le sujet de discussion de la section suivante

## Le code de production n'est pas le seul à évoluer

Nous n'arrêtons pas de dire que les tests unitaires sont aussi importants que le code de production. Et si nous faisons assez attention au code de production pour créer la meilleure abstraction possible, pourquoi ne ferions-nous pas de même pour les tests unitaires ?

Si le code des tests unitaires est aussi important que le code principal, il est alors sage de le concevoir en gardant à l'esprit l'extensibilité et de le rendre aussi maintenable que possible. Après tout, c'est le code qui devra être maintenu par un ingénieur autre que son auteur original, il doit donc être lisible.

La raison pour laquelle nous accordons autant d'attention à la flexibilité du code est que nous savons que les exigences changent et évoluent avec le temps, et finalement, à mesure que les règles métier du domaine changent, notre code devra également changer pour prendre en charge ces nouvelles exigences. Étant donné que le code de production a changé pour prendre en charge de nouvelles exigences, à son tour, le code de test devra également changer pour prendre en charge la nouvelle version du code de production.

Dans l'un des premiers exemples que nous avons utilisés, nous avons créé une série de tests pour l'objet de demande de fusion, en essayant différentes combinaisons et en vérifiant l'état auquel la demande de fusion a été laissée. C'est une bonne première approche, mais nous pouvons faire mieux que cela.


Une fois que nous comprenons mieux le problème, nous pouvons commencer à créer de meilleures abstractions. Avec cela, la première idée qui vient à l'esprit est que nous pouvons créer une abstraction de niveau supérieur qui vérifie des conditions particulières. Par exemple, si nous avons un objet qui est une suite de tests qui cible spécifiquement la classe MergeRequest, nous savons que sa fonctionnalité sera limitée au comportement de cette classe (car elle doit être conforme au SRP), et donc nous pourrions créer méthodes de test spécifiques sur cette classe de test. Ceux-ci n'auront de sens que pour cette classe, mais cela sera utile pour réduire beaucoup de code passe-partout.

Au lieu de répéter des assertions qui suivent exactement la même structure, nous pouvons créer une méthode qui encapsule cela et la réutilise dans tous les tests

In [ ]:
from unittest import TestCase



class TestMergeRequestStatus(TestCase):
    def setUp(self):
        self.merge_request = MergeRequest()

    def assert_rejected(self):
        self.assertEqual(
            self.merge_request.status, MergeRequestExtendedStatus.REJECTED
        )

    def assert_pending(self):
        self.assertEqual(
            self.merge_request.status, MergeRequestExtendedStatus.PENDING
        )

    def assert_approved(self):
        self.assertEqual(
            self.merge_request.status, MergeRequestExtendedStatus.APPROVED
        )

    def test_simple_rejected(self):
        self.merge_request.downvote("maintainer")
        self.assert_rejected()

    def test_just_created_is_pending(self):
        self.assert_pending()

    def test_pending_awaiting_review(self):
        self.merge_request.upvote("core-dev")
        self.assert_pending()

    def test_approved(self):
        self.merge_request.upvote("dev1")
        self.merge_request.upvote("dev2")
        self.assert_approved()

    def test_no_double_approve(self):
        self.merge_request.upvote("dev1")
        self.merge_request.upvote("dev1")
        self.assert_pending()

    def test_upvote_changes_to_downvote(self):
        self.merge_request.upvote("dev1")
        self.merge_request.upvote("dev2")
        self.merge_request.downvote("dev1")

        self.assert_rejected()

    def test_downvote_to_upvote(self):
        self.merge_request.upvote("dev1")
        self.merge_request.downvote("dev2")
        self.merge_request.upvote("dev2")

        self.assert_approved()


Si quelque chose change avec la façon dont nous vérifions le statut d'une demande de fusion (ou disons que nous voulons ajouter des vérifications supplémentaires), il n'y a qu'un seul endroit (la méthode assert_approved()) qui devra être modifié. Plus important encore, en créant ces abstractions de niveau supérieur, le code qui a commencé comme de simples tests unitaires commence à évoluer vers ce qui pourrait finir par être un framework de test avec sa propre API ou langage de domaine, rendant les tests plus déclaratifs

## En savoir plus sur les tests

Avec les concepts que nous avons revisités jusqu'à présent, nous savons comment tester notre code, penser notre conception en termes de comment elle va être testée et configurer les outils de notre projet pour exécuter les tests automatisés qui nous donneront un certain degré de conance concernant la qualité du logiciel que nous avons écrit.

Si notre confiance dans le code est déterminée par les tests unitaires écrits dessus, comment savons-nous qu'ils sont suffisants ? Comment pouvons-nous être sûrs que nous en avons assez fait sur les scénarios de test et que nous ne ratons pas certains tests ? Qui a dit que ces tests étaient corrects ? C'est-à-dire, qui teste les tests ?

La première partie de la question, concernant la rigueur dans les tests que nous écrivons, trouve une réponse en allant au-delà de nos efforts de test grâce à des tests basés sur les propriétés.


La deuxième partie de la question peut avoir plusieurs réponses de différents points de vue, mais nous allons brièvement mentionner les tests de mutation comme moyen de déterminer que nos tests sont effectivement corrects. En ce sens, nous pensons que les tests unitaires vérifient notre code productif principal, et cela fonctionne également comme un contrôle pour les tests unitaires


## Tests basés sur les propriétés

Les tests basés sur les propriétés consistent à générer des données pour les cas de test afin de trouver des scénarios qui feront échouer le code, qui n'étaient pas couverts par nos tests unitaires précédents.

La bibliothèque principale pour cela est l'hypothèse qui, configurée avec nos tests unitaires, nous aidera à trouver des données problématiques qui feront échouer notre code.

Nous pouvons imaginer que ce que fait cette bibliothèque est de trouver des contre-exemples pour notre code. Nous écrivons notre code de production (et les tests unitaires pour cela !), et nous prétendons qu'il est correct. Maintenant, avec cette bibliothèque, nous définissons une hypothèse qui doit être vraie pour notre code, et s'il y a des cas où nos assertions ne sont pas vraies, l'hypothèse fournira un ensemble de données qui provoque l'erreur.

La meilleure chose à propos des tests unitaires est qu'ils nous font réfléchir davantage sur notre code de production. La meilleure chose à propos de l'hypothèse est qu'elle nous fait réfléchir davantage sur nos tests unitaires

## Tests de mutation

Nous savons que les tests sont la méthode de vérication formelle dont nous disposons pour nous assurer que notre code est correct. Et qu'est-ce qui garantit que le test est correct ? Le code de production, vous pourriez penser, et oui, d'une certaine manière, c'est correct. On peut considérer le code principal comme un contrepoids à nos tests.

Le but de l'écriture de tests unitaires est que nous nous protégeons contre les bogues et testons les scénarios d'échec que nous ne voulons pas voir se produire en production. C'est bien que les tests réussissent, mais ce serait mauvais s'ils réussissent pour de mauvaises raisons. C'est-à-dire que nous pouvons utiliser des tests unitaires comme un outil de régression automatique - si quelqu'un introduit un bogue dans le code, nous nous attendons à ce qu'au moins un de nos tests l'attrape et échoue. Si cela ne se produit pas, soit il manque un test, soit ceux que nous avons ne font pas les bons contrôles.

C'est l'idée derrière les tests de mutation. Avec un outil de test de mutation, le code sera modifié en de nouvelles versions (appelées mutants) qui sont des variations du code d'origine, mais avec une partie de sa logique altérée (par exemple, les opérateurs sont échangés, les conditions sont inversées).

Une bonne suite de tests devrait attraper ces mutants et les tuer, auquel cas cela signifie que nous pouvons nous fier aux tests. Si certains mutants survivent à l'expérience, c'est généralement un mauvais signe. Bien sûr, ce n'est pas tout à fait précis, il y a donc des états intermédiaires que nous pourrions vouloir ignorer.

Pour vous montrer rapidement comment cela fonctionne et vous permettre de vous en faire une idée pratique, nous allons utiliser une version différente du code qui calcule le statut d'une demande de fusion en fonction du nombre d'approbations et de rejets. Cette fois, nous avons changé le code pour une version simple qui, sur la base de ces chiffres, renvoie le résultat. Nous avons déplacé l'énumération avec les constantes pour les statuts dans un module séparé afin qu'il semble maintenant plus compact

In [ ]:
def evaluate_merge_request(upvote_count, downvotes_count):    
  if downvotes_count > 0:        
    return Status.REJECTED    
  
  if upvote_count >= 2:        
    return Status.APPROVED    
  
  return Status.PENDING

Et maintenant allons-nous ajouter un test unitaire simple, vérifiant l'une des conditions et son résultat attendu

In [ ]:
class TestMergeRequestEvaluation(unittest.TestCase):    
  def test_approved(self):        
    result = evaluate_merge_request(3, 0)        
    self.assertEqual(result, Status.APPROVED)

Maintenant, nous allons installer mutpy, un outil de test de mutation pour Python, avec pip install mutpy, et lui dire d'exécuter le test de mutation pour ce module avec ces tests. Le code suivant s'exécute pour différents cas, qui se distinguent par la modification de la variable d'environnement CASE

      PYTHONPATH=src mut.py \
          --target src/mutation_testing_${CASE}.py \    
          --unit-test tests/test_mutation_testing_${CASE}.py \    
          --operator AOD `# delete arithmetic operator`\    
          --operator AOR `# replace arithmetic operator` \    
          --operator COD `# delete conditional operator` \    
          --operator COI `# insert conditional operator` \    
          --operator CRP `# replace constant` \    
          --operator ROR `# replace relational operator` \    
          --show-mutants

